In [ ]:
# 32345678

In [6]:
import pandas as pd
import re
from datetime import datetime, timedelta
import numpy as np

# ============================================================================
# 配置参数
# ============================================================================
file_name = '/Volumes/T7/Obentech/跨所临时表格0213.xlsx'
current_date_str = '2026/02/13'
output_file = '/Volumes/T7/Obentech_code_20251215/周报_0213.xlsx'

# USDT账户列表
usdt_accounts = ['dcPro1', 'dcPro2', 'dcPro3', 'dcPro4', 'dcPro5', 'dcPro7', 
                 'dcPro8', 'dcPro9', 'dcPro10', 'dcPro15', 'dcPro16', 'dcPro17', 'dcPro30']

# USDT托管账户（其余为公司内部）
usdt_managed_accounts = {'dcPro10', 'dcPro30'}

# 币本位账户配置
coin_based_accounts = {
    # 公司内部
    'dcPro12': {'type': '公司内部', 'coin': 'BTC', 'amount': 16, 'exchange': 'Binance-OKX'},
    'dcPro24': {'type': '公司内部', 'coin': 'BTC', 'amount': 43, 'exchange': 'Binance-OKX'},
    'dcPro26': {'type': '公司内部', 'coin': 'BTC', 'amount': 24, 'exchange': 'Binance-OKX'},
    'dcPro13': {'type': '公司内部', 'coin': 'ETH', 'amount': 1083, 'exchange': 'Binance-OKX'},
    'dcPro27': {'type': '公司内部', 'coin': 'ETH', 'amount': 352, 'exchange': 'Binance-OKX'},
    'dcPro29': {'type': '公司内部', 'coin': 'ETH', 'amount': 300, 'exchange': 'Binance-OKX'},
    
    # 托管 - BTC
    'dcPro11': {'type': '托管', 'coin': 'BTC', 'amount': 10, 'exchange': 'Binance-OKX'},
    'dcPro14': {'type': '托管', 'coin': 'BTC', 'amount': 30, 'exchange': 'Binance-OKX'},
    'dcPro38': {'type': '托管', 'coin': 'BTC', 'amount': 30, 'exchange': 'Binance-OKX'},
    'dcPro6': {'type': '托管', 'coin': 'BTC', 'amount': 23, 'exchange': 'Binance-OKX'},
    'dcPro20': {'type': '托管', 'coin': 'BTC', 'amount': 26, 'exchange': 'Binance-OKX'},
    'dcPro21': {'type': '托管', 'coin': 'BTC', 'amount': 24, 'exchange': 'Binance-OKX'},
    'dcPro31': {'type': '托管', 'coin': 'BTC', 'amount': 10, 'exchange': 'Binance-OKX'},
    'dcPro33': {'type': '托管', 'coin': 'BTC', 'amount': 13, 'exchange': 'Binance-OKX'},
    'dcPro35': {'type': '托管', 'coin': 'BTC', 'amount': 12, 'exchange': 'Binance-OKX'},
    
    # 托管 - ETH
    'dcPro18': {'type': '托管', 'coin': 'ETH', 'amount': 641, 'exchange': 'Binance-OKX'},
    'dcPro19': {'type': '托管', 'coin': 'ETH', 'amount': 554, 'exchange': 'Binance-OKX'},
    'dcPro25': {'type': '托管', 'coin': 'ETH', 'amount': 515, 'exchange': 'Binance-OKX'},
    'dcPro28': {'type': '托管', 'coin': 'ETH', 'amount': 600, 'exchange': 'Binance-OKX'},
    'dcPro32': {'type': '托管', 'coin': 'ETH', 'amount': 500, 'exchange': 'Binance-OKX'},
    
    # Binance-Bybit
    'dcbb1': {'type': '公司内部', 'coin': 'BTC', 'amount': 15, 'exchange': 'Binance-Bybit'},
}

# Gate单所套利账户
gate_accounts = ['gtpro1']

# ============================================================================
# 日期计算 - 包括今天在内的7天
# ============================================================================
current_date = datetime.strptime(current_date_str, '%Y/%m/%d')
# 7天前的第1天 = 今天 - 7天
seven_days_ago = current_date - timedelta(days=7)
seven_days_ago_str = seven_days_ago.strftime('%Y/%m/%d')
week_start_str = seven_days_ago_str

# ============================================================================
# 读取Excel文件
# ============================================================================
xls = pd.ExcelFile(file_name)

# ============================================================================
# 获取币价信息（从第一个BTC和ETH账户中读取）
# ============================================================================
btc_price = None
eth_price = None

# 尝试从第一个BTC账户获取BTC价格
for account, info in coin_based_accounts.items():
    if info['coin'] == 'BTC' and btc_price is None:
        for sheet_name in xls.sheet_names:
            if account.lower() in sheet_name.lower():
                df_temp = pd.read_excel(xls, sheet_name=sheet_name, header=0)
                df_temp.columns = df_temp.columns.str.strip()
                if 'BTC价格' in df_temp.columns and '日期' in df_temp.columns:
                    df_temp['日期'] = pd.to_datetime(df_temp['日期'], errors='coerce').dt.strftime('%Y/%m/%d')
                    current_row = df_temp[df_temp['日期'] == current_date_str]
                    if not current_row.empty:
                        btc_price = current_row.iloc[0].get('BTC价格', None)
                        break
        if btc_price:
            break

# 尝试从第一个ETH账户获取ETH价格
for account, info in coin_based_accounts.items():
    if info['coin'] == 'ETH' and eth_price is None:
        for sheet_name in xls.sheet_names:
            if account.lower() in sheet_name.lower():
                df_temp = pd.read_excel(xls, sheet_name=sheet_name, header=0)
                df_temp.columns = df_temp.columns.str.strip()
                if 'ETH价格' in df_temp.columns and '日期' in df_temp.columns:
                    df_temp['日期'] = pd.to_datetime(df_temp['日期'], errors='coerce').dt.strftime('%Y/%m/%d')
                    current_row = df_temp[df_temp['日期'] == current_date_str]
                    if not current_row.empty:
                        eth_price = current_row.iloc[0].get('ETH价格', None)
                        break
        if eth_price:
            break

print(f"BTC价格: ${btc_price:,.2f}" if btc_price else "BTC价格: 未获取")
print(f"ETH价格: ${eth_price:,.2f}" if eth_price else "ETH价格: 未获取")
print()

# ============================================================================
# 第一部分：处理USDT账户周报
# ============================================================================
print("="*80)
print("USDT账户周报")
print("="*80)

# 找到USDT账户的sheet
usdt_sheet_mapping = {}
for sheet_name in xls.sheet_names:
    match = re.match(r'(dcPro\d+)', sheet_name, re.IGNORECASE)
    if match:
        account = match.group(1)
        if account in usdt_accounts:
            usdt_sheet_mapping[account] = sheet_name

# 处理USDT账户的函数
def process_usdt_sheet(sheet_name):
    df = pd.read_excel(xls, sheet_name=sheet_name, header=0)
    df.columns = df.columns.str.strip()
    
    # 如果没有日期列，跳过此sheet
    if '日期' not in df.columns:
        return None
    
    df['日期'] = pd.to_datetime(df['日期'], errors='coerce').dt.strftime('%Y/%m/%d')
    df = df.dropna(subset=['日期'])
    
    # 周数据窗口
    week_df = df[(df['日期'] > week_start_str) & (df['日期'] <= current_date_str)]
    
    # 周总收入
    weekly_total_income = week_df['日收益'].sum() if '日收益' in week_df.columns else 'N/A'
    
    # 当前行数据
    current_row = df[df['日期'] == current_date_str]
    if current_row.empty:
        return None
    current_row = current_row.iloc[0]
    
    # 基本信息
    principal_size = current_row.get('本金', 'N/A')
    run_days = current_row.get('运行天数', 'N/A')
    current_net_value = current_row.get('净值', 'N/A')
    asset_to_principal = current_row.get('账户资产/本金', 'N/A')
    cumulative_annualized = current_row.get('累积年化收益率', 'N/A')
    weekly_annualized = current_row.get('内部过去7日平均年化', 0) * 100 if '内部过去7日平均年化' in current_row else 'N/A'
    
    # 本金变化
    principal_change = week_df['本金部分变化'].sum() if '本金部分变化' in week_df.columns else 'N/A'
    
    # 盈利变化：直接读取"盈利部分变化"列
    profit_change = week_df['盈利部分变化'].sum() if '盈利部分变化' in week_df.columns else 'N/A'
    
    # 周净值总收入：总资产变化 - 本金部分变化 - 盈利部分变化
    seven_ago_row = df[df['日期'] == seven_days_ago_str]
    net_value_income = 'N/A'
    
    if not seven_ago_row.empty:
        seven_ago_row = seven_ago_row.iloc[0]
        total_asset_change = current_row.get('总资产', 0) - seven_ago_row.get('总资产', 0)
        # 周净值总收入 = 总资产变化 - 本金部分变化 - 盈利部分变化
        if isinstance(principal_change, (int, float)) and isinstance(profit_change, (int, float)):
            net_value_income = total_asset_change - principal_change - profit_change
        else:
            net_value_income = total_asset_change
    
    # 计算周净值年化收益率：周净值总收入/本金大小*365/7*100%
    if isinstance(net_value_income, (int, float)) and isinstance(principal_size, (int, float)) and principal_size != 0:
        weekly_net_annualized = (net_value_income / principal_size) * (365 / 7) * 100
    else:
        weekly_net_annualized = 'N/A'
    
    # 其他损益
    other_col = '其他损益' if '其他损益' in week_df.columns else '其他收入' if '其他收入' in week_df.columns else None
    other_loss_profit = week_df[other_col].sum() if other_col else 'N/A'
    
    return {
        '周总收入': weekly_total_income,
        '周净值总收入': net_value_income,
        '本金大小': principal_size,
        '本金变化': principal_change,
        '盈利变化': profit_change,
        '其他损益': other_loss_profit,
        '周年化收益率': weekly_annualized,
        '周净值年化收益率': weekly_net_annualized,
        '当前净值': current_net_value,
        '账户资产/本金': asset_to_principal,
        '运行天数': run_days,
        '累计净值年化收益率': cumulative_annualized,
    }

# 处理Gate账户的函数（格式不同）
def process_gate_sheet(sheet_name):
    df = pd.read_excel(xls, sheet_name=sheet_name, header=0)
    df.columns = df.columns.str.strip()
    
    # 如果没有日期列，跳过此sheet
    if '日期' not in df.columns:
        return None
    
    df['日期'] = pd.to_datetime(df['日期'], errors='coerce').dt.strftime('%Y/%m/%d')
    df = df.dropna(subset=['日期'])
    
    # 周数据窗口
    week_df = df[(df['日期'] > week_start_str) & (df['日期'] <= current_date_str)]
    
    # 周总收入：对Gate表格使用"收益"列
    weekly_total_income = week_df['收益'].sum() if '收益' in week_df.columns else 'N/A'
    
    # 当前行数据
    current_row = df[df['日期'] == current_date_str]
    if current_row.empty:
        return None
    current_row = current_row.iloc[0]
    
    # 基本信息
    principal_size = current_row.get('本金', 'N/A')
    current_net_value = current_row.get('净值', 'N/A')
    cumulative_annualized = current_row.get('累积年化收益率', 'N/A')
    
    # 周年化收益率：Gate表格的"过去7天年化"已经是比例形式，需要乘以100转成百分比
    # 但最后一行可能是NaN，需要用前一天的值或者用周平均计算
    weekly_annualized = 'N/A'
    if '过去7天年化' in df.columns:
        # 尝试获取最近一个非NaN的"过去7天年化"值
        past_7d_col = week_df['过去7天年化'].dropna()
        if not past_7d_col.empty:
            weekly_annualized = past_7d_col.iloc[-1] * 100  # 转换为百分比
    
    # 运行天数：通过数据行数计算
    run_days = len(df)
    
    # 周净值总收入：没有总资产列，用收益累加
    net_value_income = weekly_total_income
    
    # Gate表格没有这些字段
    principal_change = 0.0
    profit_change = weekly_total_income if isinstance(weekly_total_income, (int, float)) else 'N/A'
    other_loss_profit = 0.0
    
    # 账户资产/本金：可以用净值代替
    asset_to_principal = current_net_value
    
    # 周净值年化收益率：Gate表格没有这个字段，设为N/A
    weekly_net_annualized = 'N/A'
    
    return {
        '周总收入': weekly_total_income,
        '周净值总收入': net_value_income,
        '本金大小': principal_size,
        '本金变化': principal_change,
        '盈利变化': profit_change,
        '其他损益': other_loss_profit,
        '周年化收益率': weekly_annualized,
        '周净值年化收益率': weekly_net_annualized,
        '当前净值': current_net_value,
        '账户资产/本金': asset_to_principal,
        '运行天数': run_days,
        '累计净值年化收益率': cumulative_annualized,
    }

# 处理所有USDT账户
usdt_results = []
for account in usdt_accounts:
    if account in usdt_sheet_mapping:
        result = process_usdt_sheet(usdt_sheet_mapping[account])
        if result:
            result['账号名'] = account
            usdt_results.append(result)

# 创建USDT账户DataFrame
df_usdt_report = pd.DataFrame(usdt_results)
if not df_usdt_report.empty:
    df_usdt_report = df_usdt_report[['账号名', '周总收入', '周净值总收入', '本金大小', '本金变化', '盈利变化', 
                                      '其他损益', '周年化收益率', '周净值年化收益率', '当前净值', '账户资产/本金', 
                                      '运行天数', '累计净值年化收益率']]
    print(df_usdt_report.to_string(index=False))
else:
    print("无USDT账户数据")

# ============================================================================
# 第二部分：处理Gate单所套利账户
# ============================================================================
print("\n" + "="*80)
print("Gate单所套利账户")
print("="*80)

# 找到Gate账户的sheet
gate_sheet_mapping = {}
for sheet_name in xls.sheet_names:
    match = re.match(r'(gtpro\d+)', sheet_name, re.IGNORECASE)
    if match:
        account = match.group(1)
        if account in gate_accounts:
            gate_sheet_mapping[account] = sheet_name

# 处理Gate账户
gate_results = []
for account in gate_accounts:
    if account in gate_sheet_mapping:
        result = process_gate_sheet(gate_sheet_mapping[account])
        if result:
            result['账号名'] = account
            gate_results.append(result)

# 创建Gate账户DataFrame
df_gate_report = pd.DataFrame(gate_results)
if not df_gate_report.empty:
    df_gate_report = df_gate_report[['账号名', '周总收入', '周净值总收入', '本金大小', '本金变化', '盈利变化', 
                                      '其他损益', '周年化收益率', '周净值年化收益率', '当前净值', '账户资产/本金', 
                                      '运行天数', '累计净值年化收益率']]
    print(df_gate_report.to_string(index=False))
else:
    print("无Gate账户数据")

# ============================================================================
# 第三部分：处理币本位账户周报
# ============================================================================
print("\n" + "="*80)
print("币本位账户周报")
print("="*80)

# 找到币本位账户的sheet
coin_sheet_mapping = {}
for sheet_name in xls.sheet_names:
    # 匹配dcPro或dcbb开头的账户
    match = re.match(r'(dcPro\d+|dcbb\d+)', sheet_name, re.IGNORECASE)
    if match:
        account = match.group(1)
        if account in coin_based_accounts:
            coin_sheet_mapping[account] = sheet_name

# 处理币本位账户的函数 - 返回详细信息
def process_coin_sheet_detail(sheet_name, account_name, coin_type, coin_amount, account_type, exchange):
    df = pd.read_excel(xls, sheet_name=sheet_name, header=0)
    df.columns = df.columns.str.strip()
    
    # 如果没有日期列，跳过此sheet
    if '日期' not in df.columns:
        return None
    
    df['日期'] = pd.to_datetime(df['日期'], errors='coerce').dt.strftime('%Y/%m/%d')
    df = df.dropna(subset=['日期'])
    
    # 周数据窗口
    week_df = df[(df['日期'] > week_start_str) & (df['日期'] <= current_date_str)]
    
    current_row = df[df['日期'] == current_date_str]
    if current_row.empty:
        return None
    current_row = current_row.iloc[0]
    
    # 周总收入（币单位）
    weekly_total_income = week_df['日收益'].sum() if '日收益' in week_df.columns else 0
    
    # 周净值总收入（币单位）
    seven_ago_row = df[df['日期'] == seven_days_ago_str]
    net_value_income = 0
    principal_change = week_df['本金部分变化'].sum() if '本金部分变化' in week_df.columns else 0
    
    if not seven_ago_row.empty and '总资产' in df.columns:
        seven_ago_row = seven_ago_row.iloc[0]
        net_value_income = current_row.get('总资产', 0) - seven_ago_row.get('总资产', 0) - principal_change
    
    # 本金（币的数量）
    principal_coin = current_row.get('本金', coin_amount)
    
    # 周年化收益率
    weekly_annualized = current_row.get('内部过去7日平均年化', 0) * 100 if '内部过去7日平均年化' in current_row else 0
    
    # 累计年化收益率
    cumulative_annualized = current_row.get('累积年化收益率', 0)
    
    # 转换为USDT
    coin_price = btc_price if coin_type == 'BTC' else eth_price
    weekly_total_income_usdt = weekly_total_income * coin_price if coin_price else 0
    net_value_income_usdt = net_value_income * coin_price if coin_price else 0
    
    return {
        '账号名': account_name,
        '类型': account_type,
        '交易所': exchange,
        '币种': coin_type,
        '数量': coin_amount,
        '本金(币)': principal_coin,
        '周总收入(币)': weekly_total_income,
        '周净值总收入(币)': net_value_income,
        '周总收入(USDT)': weekly_total_income_usdt,
        '周净值总收入(USDT)': net_value_income_usdt,
        '周年化收益率': weekly_annualized,
        '周年化收益率(%)': f"{weekly_annualized:.2f}" if isinstance(weekly_annualized, (int, float)) else 'N/A',
        '累计年化收益率(%)': f"{cumulative_annualized*100:.2f}" if isinstance(cumulative_annualized, (int, float)) else 'N/A',
    }

# 处理所有币本位账户 - 保存详细信息
coin_results_detail = []
for account, info in coin_based_accounts.items():
    if account in coin_sheet_mapping:
        result = process_coin_sheet_detail(
            coin_sheet_mapping[account], 
            account,
            info['coin'], 
            info['amount'],
            info['type'],
            info['exchange']
        )
        if result:
            coin_results_detail.append(result)

# 创建币本位账户DataFrame（用于显示）
if coin_results_detail:
    df_coin_report_display = pd.DataFrame([
        {
            '账号名': r['账号名'],
            '类型': r['类型'],
            '交易所': r['交易所'],
            '币种': r['币种'],
            '数量': r['数量'],
            '周总收入(币)': f"{r['周总收入(币)']:.4f}",
            '周净值总收入(币)': f"{r['周净值总收入(币)']:.4f}",
            '周年化收益率(%)': r['周年化收益率(%)'],
            '累计年化收益率(%)': r['累计年化收益率(%)']
        }
        for r in coin_results_detail
    ])
    df_coin_report_display = df_coin_report_display.sort_values(['交易所', '币种', '类型', '账号名']).reset_index(drop=True)
    print(df_coin_report_display.to_string(index=False))
else:
    df_coin_report_display = pd.DataFrame()
    print("无币本位账户数据")

# ============================================================================
# 第四部分：计算实盘总体情况统计
# ============================================================================
print("\n" + "="*80)
print("实盘总体情况统计")
print("="*80)

# 辅助函数：计算加权平均、范围、总和
def calculate_summary_stats(df, principal_col, weekly_ann_col, weekly_income_col, weekly_net_income_col):
    """
    df: DataFrame包含账户数据
    principal_col: 本金列名
    weekly_ann_col: 周年化收益率列名
    weekly_income_col: 周总收入列名（USDT）
    weekly_net_income_col: 周净值总收入列名（USDT）
    """
    # 过滤掉无效数据
    valid_df = df.copy()
    valid_df = valid_df[pd.notna(valid_df[principal_col]) & (valid_df[principal_col] != 'N/A')]
    valid_df = valid_df[pd.notna(valid_df[weekly_ann_col]) & (valid_df[weekly_ann_col] != 'N/A')]
    
    if valid_df.empty:
        return None
    
    # 转换为数值
    valid_df[principal_col] = pd.to_numeric(valid_df[principal_col], errors='coerce')
    valid_df[weekly_ann_col] = pd.to_numeric(valid_df[weekly_ann_col], errors='coerce')
    valid_df[weekly_income_col] = pd.to_numeric(valid_df[weekly_income_col], errors='coerce')
    valid_df[weekly_net_income_col] = pd.to_numeric(valid_df[weekly_net_income_col], errors='coerce')
    
    # 去除NaN
    valid_df = valid_df.dropna(subset=[principal_col, weekly_ann_col])
    
    if valid_df.empty:
        return None
    
    # 计算统计数据
    total_principal = valid_df[principal_col].sum()
    weighted_avg = (valid_df[principal_col] * valid_df[weekly_ann_col]).sum() / total_principal if total_principal > 0 else 0
    
    min_ann = valid_df[weekly_ann_col].min()
    max_ann = valid_df[weekly_ann_col].max()
    
    total_income = valid_df[weekly_income_col].sum() if weekly_income_col in valid_df.columns else 0
    total_net_income = valid_df[weekly_net_income_col].sum() if weekly_net_income_col in valid_df.columns else 0
    
    # 收益率范围
    if len(valid_df) == 1:
        ann_range = f"{min_ann:.2f}%"
    else:
        ann_range = f"{min_ann:.2f}% ~ {max_ann:.2f}%"
    
    return {
        'weighted_avg': weighted_avg,
        'ann_range': ann_range,
        'total_income': total_income,
        'total_net_income': total_net_income,
        'total_principal': total_principal
    }

# 创建汇总列表
summary_rows = []

# 1. Binance-OKX跨所 USDT - 分为公司内部和托管
if not df_usdt_report.empty:
    # USDT 公司内部
    df_usdt_internal = df_usdt_report[~df_usdt_report['账号名'].isin(usdt_managed_accounts)]
    if not df_usdt_internal.empty:
        usdt_internal_stats = calculate_summary_stats(df_usdt_internal, '本金大小', '周年化收益率', '周总收入', '周净值总收入')
        if usdt_internal_stats:
            summary_rows.append({
                '类型': 'Binance-OKX跨所 USDT 公司内部',
                '周年化收益率范围': usdt_internal_stats['ann_range'],
                '资金加权周年化': f"{usdt_internal_stats['weighted_avg']:.2f}%",
                '周总收入(USDT)': f"{usdt_internal_stats['total_income']:,.2f}",
                '周净值总收入(USDT)': f"{usdt_internal_stats['total_net_income']:,.2f}"
            })
    
    # USDT 托管
    df_usdt_managed = df_usdt_report[df_usdt_report['账号名'].isin(usdt_managed_accounts)]
    if not df_usdt_managed.empty:
        usdt_managed_stats = calculate_summary_stats(df_usdt_managed, '本金大小', '周年化收益率', '周总收入', '周净值总收入')
        if usdt_managed_stats:
            summary_rows.append({
                '类型': 'Binance-OKX跨所 USDT 托管',
                '周年化收益率范围': usdt_managed_stats['ann_range'],
                '资金加权周年化': f"{usdt_managed_stats['weighted_avg']:.2f}%",
                '周总收入(USDT)': f"{usdt_managed_stats['total_income']:,.2f}",
                '周净值总收入(USDT)': f"{usdt_managed_stats['total_net_income']:,.2f}"
            })

# 2-6. 币本位账户各分类
coin_categories = [
    ('Binance-OKX跨所 BTC 公司内部', 'Binance-OKX', 'BTC', '公司内部'),
    ('Binance-OKX跨所 BTC 托管', 'Binance-OKX', 'BTC', '托管'),
    ('Binance-OKX跨所 ETH 公司内部', 'Binance-OKX', 'ETH', '公司内部'),
    ('Binance-OKX跨所 ETH 托管', 'Binance-OKX', 'ETH', '托管'),
]

for label, exchange, coin, account_type in coin_categories:
    filtered = [r for r in coin_results_detail 
                if r['交易所'] == exchange and r['币种'] == coin and r['类型'] == account_type]
    
    if filtered:
        df_filtered = pd.DataFrame(filtered)
        coin_stats = calculate_summary_stats(df_filtered, '本金(币)', '周年化收益率', '周总收入(USDT)', '周净值总收入(USDT)')
        
        if coin_stats:
            summary_rows.append({
                '类型': label,
                '周年化收益率范围': coin_stats['ann_range'],
                '资金加权周年化': f"{coin_stats['weighted_avg']:.2f}%",
                '周总收入(USDT)': f"{coin_stats['total_income']:,.2f}",
                '周净值总收入(USDT)': f"{coin_stats['total_net_income']:,.2f}"
            })

# Binance-Bybit跨所 BTC
btc_bn_bybit = [r for r in coin_results_detail 
                if r['交易所'] == 'Binance-Bybit' and r['币种'] == 'BTC']
if btc_bn_bybit:
    df_bybit = pd.DataFrame(btc_bn_bybit)
    bybit_stats = calculate_summary_stats(df_bybit, '本金(币)', '周年化收益率', '周总收入(USDT)', '周净值总收入(USDT)')
    
    if bybit_stats:
        summary_rows.append({
            '类型': 'Binance-Bybit跨所 BTC',
            '周年化收益率范围': bybit_stats['ann_range'],
            '资金加权周年化': f"{bybit_stats['weighted_avg']:.2f}%",
            '周总收入(USDT)': f"{bybit_stats['total_income']:,.2f}",
            '周净值总收入(USDT)': f"{bybit_stats['total_net_income']:,.2f}"
        })

# 7. Gate单所套利 USDT
if not df_gate_report.empty:
    gate_stats = calculate_summary_stats(df_gate_report, '本金大小', '周年化收益率', '周总收入', '周净值总收入')
    if gate_stats:
        summary_rows.append({
            '类型': 'Gate单所套利 USDT',
            '周年化收益率范围': gate_stats['ann_range'],
            '资金加权周年化': f"{gate_stats['weighted_avg']:.2f}%",
            '周总收入(USDT)': f"{gate_stats['total_income']:,.2f}",
            '周净值总收入(USDT)': f"{gate_stats['total_net_income']:,.2f}"
        })

# 创建汇总表
df_summary = pd.DataFrame(summary_rows)
if not df_summary.empty:
    print(df_summary.to_string(index=False))
else:
    print("无统计数据")

# ============================================================================
# 输出到Excel文件
# ============================================================================
print("\n" + "="*80)
print(f"正在保存到: {output_file}")
print("="*80)

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    if not df_usdt_report.empty:
        df_usdt_report.to_excel(writer, sheet_name='USDT账户周报', index=False)
    if not df_gate_report.empty:
        df_gate_report.to_excel(writer, sheet_name='Gate单所套利', index=False)
    if not df_coin_report_display.empty:
        df_coin_report_display.to_excel(writer, sheet_name='币本位账户周报', index=False)
    if not df_summary.empty:
        df_summary.to_excel(writer, sheet_name='实盘总体情况统计', index=False)

print(f"✓ Excel文件已保存: {output_file}")
if not df_usdt_report.empty:
    print(f"  - Sheet 1: USDT账户周报 ({len(df_usdt_report)} 个账户)")
if not df_gate_report.empty:
    print(f"  - Sheet 2: Gate单所套利 ({len(df_gate_report)} 个账户)")
if not df_coin_report_display.empty:
    print(f"  - Sheet 3: 币本位账户周报 ({len(df_coin_report_display)} 个账户)")
if not df_summary.empty:
    print(f"  - Sheet 4: 实盘总体情况统计")
print(f"\n说明: 使用币价 - BTC: ${btc_price:,.2f}, ETH: ${eth_price:,.2f}" if btc_price and eth_price else "")

BTC价格: $66,336.29
ETH价格: $1,942.30

USDT账户周报
    账号名        周总收入        周净值总收入         本金大小      本金变化    盈利变化 其他损益    周年化收益率     周净值年化收益率     当前净值  账户资产/本金  运行天数 累计净值年化收益率
 dcPro1 2292.271598  2.563827e+03 1000000.0000       0.0     0.0  0.0 11.952559          N/A 1.076519 1.012474 240.0  0.118663
 dcPro2 4923.237355  5.820435e+03 2033597.2672       0.0     0.0  N/A 12.623525    14.924003 1.051459 0.994994 200.0    0.0959
 dcPro3 6037.923571  6.545658e+03 3050000.0000 -300000.0 -2199.0  0.0 10.322445          N/A 1.056597 1.009155 207.0  0.101943
 dcPro4 5571.755384  8.497326e+03 2700000.0000       0.0     0.0  0.0 10.760268          N/A 1.054797 1.013040 200.0  0.102259
 dcPro5 1697.871375  5.157509e+04  653319.0000       0.0     0.0  0.0 13.551093   411.632342 1.081783 0.915544 197.0  0.156791
 dcPro7 4501.518105  5.118404e+03 2050000.0000 -300000.0 -2778.0  0.0 11.318006          N/A 1.042272 1.011645 179.0  0.088091
 dcPro8 3460.530721  2.361578e+03 2036511.6000       0.0     0.0  

In [19]:
import pandas as pd
import re
from datetime import datetime, timedelta

# Load the Excel file
file_name = '/Users/rayxu/Downloads/跨所临时表格 (1).xlsx'
xls = pd.ExcelFile(file_name)

# List of target accounts
target_accounts = ['dcPro1', 'dcPro2', 'dcPro3', 'dcPro4', 'dcPro5', 'dcPro7', 'dcPro8', 'dcPro9', 'dcPro10', 'dcPro15', 'dcPro16', 'dcPro17']

# Find matching sheet names
sheet_mapping = {}
for sheet_name in xls.sheet_names:
    match = re.match(r'(dcPro\d+)', sheet_name, re.IGNORECASE)
    if match:
        account = match.group(1)
        if account in target_accounts:
            sheet_mapping[account] = sheet_name

# Current date (change as needed)
current_date_str = '2026/01/29'
current_date = datetime.strptime(current_date_str, '%Y/%m/%d')
seven_days_ago = current_date - timedelta(days=7)
seven_days_ago_str = seven_days_ago.strftime('%Y/%m/%d')
week_start_str = (current_date - timedelta(days=6)).strftime('%Y/%m/%d')  # 7 days including current

# Function to process each sheet
def process_sheet(sheet_name):
    df = pd.read_excel(xls, sheet_name=sheet_name, header=0)
    
    # Clean column names
    df.columns = df.columns.str.strip()
    
    # Convert date to string
    df['日期'] = pd.to_datetime(df['日期'], errors='coerce').dt.strftime('%Y/%m/%d')
    df = df.dropna(subset=['日期'])
    
    # Component columns for daily income
    component_cols = [
        '单所bn资费', '单所bnTt价差', '单所bnTm价差', '单所bn借贷',
        '单所okx资费', '单所okx价差', '单所okx质押', '单所okx借贷',
        '跨所双合约资费', '跨所双合约价差', '单所双合约资费', '单所双合约价差'
    ]
    other_col = '其他损益' if '其他损益' in df.columns else '其他收入' if '其他收入' in df.columns else None
    if other_col:
        component_cols += [other_col]
    
    df['calculated_daily_income'] = df[component_cols].sum(axis=1, skipna=True)
    
    # Fill forward本金
    if '本金' in df.columns:
        df['本金'] = df['本金'].ffill()
    
    # Fill 日策略内部计算年化收益率
    if '日策略内部计算年化收益率' in df.columns:
        df['日策略内部计算年化收益率'] = df['日策略内部计算年化收益率'].fillna((df['calculated_daily_income'] / df['本金']) * 365)
    
    # Fill 总资产
    if '总资产' in df.columns:
        last_total_idx = df['总资产'].last_valid_index()
        if last_total_idx is not None:
            last_total = df.at[last_total_idx, '总资产']
            principal_ch_col = '本金部分变化' if '本金部分变化' in df.columns else '新增本金' if '新增本金' in df.columns else None
            for i in range(last_total_idx + 1, len(df)):
                daily_inc = df.at[i, 'calculated_daily_income']
                principal_ch = df.at[i, principal_ch_col] if principal_ch_col else 0
                last_total += daily_inc + principal_ch
                df.at[i, '总资产'] = last_total
    
    # Fill 净值 and 账户资产/本金
    if '净值' in df.columns and '总资产' in df.columns and '本金' in df.columns:
        df['净值'] = df['净值'].fillna(df['总资产'] / df['本金'])
    if '账户资产/本金' in df.columns and '总资产' in df.columns and '本金' in df.columns:
        df['账户资产/本金'] = df['账户资产/本金'].fillna(df['总资产'] / df['本金'])
    
    # Filter for the week
    week_df = df[(df['日期'] >= week_start_str) & (df['日期'] <= current_date_str)]
    
    # 周总收入: sum '日收益' or calculated
    if '日收益' in week_df.columns:
        weekly_total_income = week_df['日收益'].fillna(week_df['calculated_daily_income']).sum()
    else:
        weekly_total_income = week_df['calculated_daily_income'].sum()
    
    # Current row
    current_row = df[df['日期'] == current_date_str]
    if current_row.empty:
        return {k: 'N/A' for k in ['周总收入', '周净值总收入', '本金大小', '本金变化', '盈利变化', '其他损益', '周年化收益率', '周净值年化收益率', '当前净值', '账户资产/本金', '运行天数', '累计净值年化收益率']}
    current_row = current_row.iloc[0]
    
    # 本金大小
    principal_size = current_row.get('本金', 'N/A')
    
    # 运行天数
    run_days = current_row.get('运行天数', len(df))
    
    # 当前净值
    current_net_value = current_row.get('净值', 'N/A')
    
    # 账户资产/本金
    asset_to_principal = current_row.get('账户资产/本金', 'N/A')
    
    # 累计净值年化收益率
    cumulative_annualized = current_row.get('累积年化收益率', ((current_net_value - 1) / run_days * 365) if current_net_value != 'N/A' and run_days != 'N/A' else 'N/A')
    
    # 周净值年化收益率
    weekly_net_annualized = current_row.get('过去7日净值年化收益率', 'N/A')
    if weekly_net_annualized == 'N/A':
        if principal_size != 'N/A':
            avg_principal = principal_size  # approximate
            total_return = weekly_total_income / avg_principal
            weekly_net_annualized = total_return * (365 / 7)
    
    # 周年化收益率
    weekly_annualized = current_row.get('内部过去7日平均年化收益率', 'N/A') * 100 if '内部过去7日平均年化收益率' in current_row else 'N/A'
    if not isinstance(weekly_annualized, (int, float)):
        if '日策略内部计算年化收益率' in week_df.columns:
            avg_daily_annual = week_df['日策略内部计算年化收益率'].mean()
            weekly_annualized = avg_daily_annual * 100
    
    # 周净值总收入
    seven_ago_row = df[df['日期'] == seven_days_ago_str]
    net_value_income = 'N/A'
    if not seven_ago_row.empty and '总资产' in df.columns:
        seven_ago_row = seven_ago_row.iloc[0]
        net_value_income = current_row['总资产'] - seven_ago_row['总资产']
    if not isinstance(net_value_income, (int, float)):
        net_value_income = weekly_total_income  # approximate with sum income if no asset change
    
    # 本金变化
    principal_change_col = '本金部分变化' if '本金部分变化' in week_df.columns else '新增本金' if '新增本金' in week_df.columns else None
    principal_change = week_df[principal_change_col].sum() if principal_change_col else 'N/A'
    
    # 盈利变化
    profit_change_col = '盈利部分变化' if '盈利部分变化' in week_df.columns else None
    profit_change = week_df[profit_change_col].sum() if profit_change_col else weekly_total_income  # approximate
    
    # 其他损益
    other_loss_profit = week_df[other_col].sum() if other_col else 'N/A'
    
    return {
        '周总收入': weekly_total_income,
        '周净值总收入': net_value_income,
        '本金大小': principal_size,
        '本金变化': principal_change,
        '盈利变化': profit_change,
        '其他损益': other_loss_profit,
        '周年化收益率': weekly_annualized,
        '周净值年化收益率': weekly_net_annualized,
        '当前净值': current_net_value,
        '账户资产/本金': asset_to_principal,
        '运行天数': run_days,
        '累计净值年化收益率': cumulative_annualized
    }

# Process all
results = {}
for account in target_accounts:
    if account in sheet_mapping:
        results[account] = process_sheet(sheet_mapping[account])
    else:
        results[account] = {k: 'No Sheet' for k in results.keys() or ['周总收入']}  # Placeholder

# Print the table
header = '账号名 周总收入 周净值总收入 本金大小 本金变化 盈利变化 其他损益 周年化收益率 周净值年化收益率 当前净值 账户资产/本金 运行天数 累计净值年化收益率'
print(header)
for account in target_accounts:
    data = results.get(account, {})
    row = f"{account} {data.get('周总收入', 'N/A')} {data.get('周净值总收入', 'N/A')} {data.get('本金大小', 'N/A')} {data.get('本金变化', 'N/A')} {data.get('盈利变化', 'N/A')} {data.get('其他损益', 'N/A')} {data.get('周年化收益率', 'N/A')} {data.get('周净值年化收益率', 'N/A')} {data.get('当前净值', 'N/A')} {data.get('账户资产/本金', 'N/A')} {data.get('运行天数', 'N/A')} {data.get('累计净值年化收益率', 'N/A')}"
    print(row)

TypeError: can only concatenate str (not "int") to str

In [ ]:
import re
import numpy as np
import pandas as pd

import pandas as pd
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")  # 6位小数，可改
# 或者你想完全不带小数：
# pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

def _parse_date_series(s: pd.Series) -> pd.Series:
    """支持 datetime / 'YYYY/MM/DD' / 'YYYY-MM-DD' 等，统一到 date 粒度"""
    return pd.to_datetime(s, errors="coerce").dt.normalize()


def _to_number(x):
    """把各种单元格值转成 float；支持 '1,234'、'12.3%'"""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    if isinstance(x, str):
        t = x.strip()
        if t == "":
            return np.nan
        t = t.replace(",", "")
        if t.endswith("%"):
            try:
                return float(t[:-1]) / 100.0
            except:
                return np.nan
        try:
            return float(t)
        except:
            return np.nan
    return np.nan


def _choose_effective_end(df: pd.DataFrame, end: pd.Timestamp, key_cols=("总资产", "本金", "净值")) -> pd.Timestamp:
    """
    有些 sheet 可能 end_date 那行存在但关键列是空；
    这里自动回退到 <= end_date 最近一个“关键列有值”的日期。
    """
    df_le = df[df["_date"] <= end].sort_values("_date")
    if df_le.empty:
        return df.sort_values("_date").iloc[0]["_date"]
    for d in sorted(df_le["_date"].unique(), reverse=True):
        r = df_le[df_le["_date"] == d].iloc[-1]
        if any((c in df_le.columns and pd.notna(_to_number(r[c]))) for c in key_cols):
            return d
    return df_le["_date"].max()


def _compute_sheet_metrics(df: pd.DataFrame, end_date: str) -> dict:
    # 日期列：优先“日期”，否则用第一列
    date_col = "日期" if "日期" in df.columns else df.columns[0]

    df = df.copy()
    df["_date"] = _parse_date_series(df[date_col])
    df = df[df["_date"].notna()].sort_values("_date")
    if df.empty:
        return {}

    end = pd.to_datetime(end_date).normalize()
    actual_end = _choose_effective_end(df, end)
    end_row = df[df["_date"] == actual_end].iloc[-1]

    # 周窗口：实际结束日往前 6 天（共7天）
    start = actual_end - pd.Timedelta(days=5)
    win = df[(df["_date"] >= start) & (df["_date"] <= actual_end)]

    # 周净值总收入：用 end-7 的“总资产”作基准；若缺失取 <=end-7 最近一天
    base_target = actual_end - pd.Timedelta(days=6)
    base_df = df[df["_date"] <= base_target]
    base_date = base_df["_date"].max() if not base_df.empty else pd.NaT
    base_row = df[df["_date"] == base_date].iloc[-1] if pd.notna(base_date) else None

    def end_val(col):
        return _to_number(end_row[col]) if col in df.columns else np.nan

    def sum_col(col):
        if col not in df.columns:
            return np.nan
        ser = win[col].map(_to_number)
        return ser.sum(skipna=True) if ser.notna().any() else np.nan

    # 周总收入：日收益 -> 当天收入 -> 总收入
    week_total = np.nan
    for c in ["日收益"]:
        v = sum_col(c)
        if not np.isnan(v):
            week_total = v
            break

    # 周净值总收入：总资产(end) - 总资产(end-7)
    week_nav = np.nan
    if "总资产" in df.columns and base_row is not None:
        ea = _to_number(end_row["总资产"])
        ba = _to_number(base_row["总资产"])
        if not (np.isnan(ea) or np.isnan(ba)):
            week_nav = ea - ba

    principal = end_val("本金")
    principal_chg = sum_col("本金部分变化")
    profit_chg = sum_col("盈利部分变化")

    # 其他损益：优先“其他损益/其他收入”求和；否则用残差
    other = np.nan
    for c in ["其他损益", "其他收入"]:
        v = sum_col(c)
        if not np.isnan(v):
            other = v
            break
    if np.isnan(other) and not np.isnan(week_total):
        other = week_total - (0 if np.isnan(principal_chg) else principal_chg) - (0 if np.isnan(profit_chg) else profit_chg)

    ann_7d = end_val("内部过去7日平均年化")
    ann_7d_pct = ann_7d if not np.isnan(ann_7d) else np.nan  # 输出百分数

    return {
        "周总收入": week_total,
        "周净值总收入": week_nav,
        "本金大小": principal,
        "本金变化": principal_chg,
        "盈利变化": profit_chg,
        "其他损益": other,
        "周年化收益率": ann_7d_pct,
        "周净值年化收益率": end_val("过去7日净值年化收益率"),
        "当前净值": end_val("净值"),
        "账户资产/本金": end_val("账户资产/本金"),
        "运行天数": end_val("运行天数"),
        "累计净值年化收益率": end_val("累积年化收益率"),
    }


def generate_weekly_report(excel_path: str, end_date: str, accounts=None) -> pd.DataFrame:
    """
    accounts: None 或 list/set，例如 {'dcPro1','dcPro2',...}
    输出一行一个账号（按 sheet name 前缀 dcProxx 匹配）
    """
    xls = pd.ExcelFile(excel_path)
    acct_filter = set(accounts) if accounts else None

    rows = []
    for sheet in xls.sheet_names:
        m = re.match(r"^(dcPro\d+)", sheet, re.IGNORECASE)
        if not m:
            continue
        acct = "dcPro" + re.findall(r"\d+", m.group(1))[0]  # normalize: dcPro1
        if acct_filter and acct not in acct_filter:
            continue

        df = pd.read_excel(excel_path, sheet_name=sheet, engine="openpyxl")
        met = _compute_sheet_metrics(df, end_date)
        if not met:
            continue

        row = {"账号名": acct}
        row.update(met)
        rows.append(row)

    out = pd.DataFrame(rows).sort_values("账号名").reset_index(drop=True)
    out = out[
        [
            "账号名",
            "周总收入",
            "周净值总收入",
            "本金大小",
            "本金变化",
            "盈利变化",
            "其他损益",
            "周年化收益率",
            "周净值年化收益率",
            "当前净值",
            "账户资产/本金",
            "运行天数",
            "累计净值年化收益率",
        ]
    ]
    return out


df_week = generate_weekly_report(
    '/Volumes/T7/Obentech_code_20251215/跨所临时表格 (2).xlsx',
    "2026-01-30",
    accounts=["dcPro1","dcPro2","dcPro3","dcPro4","dcPro5","dcPro7","dcPro8","dcPro9","dcPro10","dcPro15","dcPro16","dcPro17"]
)
df_week


,账号名,周总收入,周净值总收入,本金大小,本金变化,盈利变化,其他损益,周年化收益率,周净值年化收益率,当前净值,账户资产/本金,运行天数,累计净值年化收益率
0,dcPro1,921.2133,20.4572,"1,000,000.0000",0.0000,0.0000,0.0000,0.0201,0.0001,1.0739,1.0100,226.0000,0.1220
1,dcPro10,"1,250.6036","1,046.1670","3,078,840.0000",0.0000,0.0000,0.0000,0.0289,0.0024,1.0366,1.0101,142.0000,0.0969
2,dcPro15,"1,111.3092","-2,010.5264","2,571,748.2200",0.0000,0.0000,0.0000,0.0106,-0.0054,1.0048,1.0048,123.0000,0.0143
3,dcPro16,"1,123.2724",958.8960,"3,297,287.0000",0.0000,0.0000,0.0000,0.0321,0.0020,1.0321,1.0060,120.0000,0.1010
4,dcPro17,"1,415.5486","1,443.0156","3,410,541.8635",0.0000,0.0000,0.0000,0.0233,0.0029,1.0329,1.0131,122.0000,0.1018
5,dcPro2,"1,626.2443","8,321.6327","2,033,597.2672",0.0000,0.0000,"1,626.2443",0.0228,0.0294,1.0484,0.9921,186.0000,0.0971
6,dcPro3,"2,394.3922",94.4121,"3,850,000.0000",0.0000,0.0000,0.0000,0.0176,0.0002,1.0539,1.0068,193.0000,0.1044
7,dcPro4,"2,074.5246",-830.0271,"2,700,000.0000",0.0000,0.0000,0.0000,0.0170,-0.0021,1.0506,1.0090,186.0000,0.1018
8,dcPro5,508.6480,"-114,478.7251","653,319.0000","-114,922.4800",0.0000,0.0000,0.0352,0.0046,0.9887,0.8368,183.0000,-0.0224
9,dcPro7,996.8036,"1,417.3923","2,350,000.0000",0.0000,0.0000,0.0000,0.0368,0.0042,1.0393,1.0087,165.0000,0.0889


In [ ]:
df_week.to_excel('/Volumes/T7/Obentech_code_20251215/stats_0130_2.xlsx')

In [ ]:
df = pd.read_excel('/Volumes/T7/Obentech_code_20251215/跨所临时表格 (1).xlsx', sheet_name="dcPro1_内部公司_100wu")
df['日收益']

0             NaN
1             NaN
2       81.103162
3      270.547530
4      306.210456
          ...    
216    166.212079
217    156.635815
218    173.443045
219    312.135238
220     72.696743
Name: 日收益, Length: 221, dtype: float64

In [ ]:
_compute_sheet_metrics(df, "2026-01-29")

{'周总收入': -221.22039115400227,
 '周净值总收入': -645.46603728435,
 '本金大小': 1000000.0,
 '本金变化': 0.0,
 '盈利变化': 0.0,
 '其他损益': 0.0,
 '周年化收益率': -1.1535063253030124,
 '周净值年化收益率': -0.004465694719349211,
 '当前净值': 1.0730242003710229,
 '账户资产/本金': 1.009187899114,
 '运行天数': 225.0,
 '累计净值年化收益率': 0.12112861798653807}

In [ ]:
import re
import numpy as np
import pandas as pd
import openpyxl
from openpyxl.utils import column_index_from_string

excel_path = "/Users/rayxu/Downloads/跨所临时表格 (1).xlsx"
sheet = "dcPro1_内部公司_100wu"

# 1) pandas 读取（会出现公式列 NaN）
df = pd.read_excel(excel_path, sheet_name=sheet, engine="openpyxl")
print("shape:", df.shape)
print("columns contains 日收益?", "日收益" in df.columns)
display(df[["日期","日收益","总收入","本金"]].head(20))

# 2) 用 openpyxl 看“日收益”列在xlsx里到底存的是什么（通常是公式）
wb = openpyxl.load_workbook(excel_path, data_only=False)  # data_only=False => 读到公式本身
ws = wb[sheet]

# 找到表头里“日收益 / 总收入 / 本金 / 日策略内部计算年化收益率”对应的列号
header = [ws.cell(1, c).value for c in range(1, 200)]
def find_col(name):
    for i, v in enumerate(header, start=1):
        if v == name:
            return i
    return None

col_rishouyi = find_col("日收益")
col_total = find_col("总收入")
col_principal = find_col("本金")
col_daily_ann = find_col("日策略内部计算年化收益率")

print("col(日收益/总收入/本金/日策略内部计算年化收益率):",
      col_rishouyi, col_total, col_principal, col_daily_ann)

# 打印前几行“日收益”单元格内容（你会看到是 =... 公式，或者 None）
for r in range(2, 25):
    v = ws.cell(r, col_rishouyi).value
    if v is not None:
        print(f"row {r} 日收益 cell raw =", v)
        break

# 3) 关键：在Python里复现Excel公式，把 NaN 的公式列算出来
def to_num(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    if isinstance(x, str):
        t = x.strip().replace(",", "")
        if t == "" or t == "\n":
            return np.nan
        if t.endswith("%"):
            try:
                return float(t[:-1]) / 100.0
            except:
                return np.nan
        try:
            return float(t)
        except:
            return np.nan
    return np.nan

# --- 3.1 先复现“总收入”的公式（解析xlsx里总收入那列的公式，自动找加减项） ---
def infer_total_income_components():
    # 找到一个“总收入”列里含公式的行，拿它的公式来解析
    for r in range(2, ws.max_row + 1):
        cell = ws.cell(r, col_total).value
        if isinstance(cell, str) and cell.startswith("="):
            formula = cell[1:]
            return formula
    return None

formula_total = infer_total_income_components()
print("总收入公式示例:", formula_total)

# 解析公式：支持两类：
# 1) B15+C15+D15-E15+...
# 2) SUM(B4:G4)-I4+SUM(J4:K4)
def parse_income_formula(formula: str):
    plus_letters = []
    minus_letters = []

    if formula is None:
        return plus_letters, minus_letters

    # 先处理 SUM(A1:B1) 这种
    sum_pat = re.compile(r'([+-]?)\s*SUM\(([A-Z]+)\d+:\s*([A-Z]+)\d+\)', re.I)
    for sign, a, b in sum_pat.findall(formula):
        sign = sign.strip()
        if sign == "-":
            target = minus_letters
        else:
            target = plus_letters
        ia = column_index_from_string(a.upper())
        ib = column_index_from_string(b.upper())
        for idx in range(min(ia, ib), max(ia, ib) + 1):
            # idx -> letter
            # openpyxl utils get_column_letter 也行，这里简单用 ws.cell(1,idx).column_letter
            target.append(ws.cell(1, idx).column_letter)

    # 去掉 SUM(...) 片段，剩下单项引用
    formula_wo_sum = sum_pat.sub("", formula)

    # 再解析形如 +B15 -E15 这种
    term_pat = re.compile(r'([+-]?)\s*([A-Z]+)\d+', re.I)
    for sign, col in term_pat.findall(formula_wo_sum):
        sign = sign.strip()
        col = col.upper()
        if sign == "-":
            minus_letters.append(col)
        else:
            plus_letters.append(col)

    return plus_letters, minus_letters

plus_letters, minus_letters = parse_income_formula(formula_total)

# 把列字母映射成df里的列名（来自header行）
def letter_to_colname(letter):
    idx = column_index_from_string(letter)
    return ws.cell(1, idx).value

plus_cols = [letter_to_colname(x) for x in plus_letters if letter_to_colname(x) in df.columns]
minus_cols = [letter_to_colname(x) for x in minus_letters if letter_to_colname(x) in df.columns]

print("总收入 = +", plus_cols)
print("总收入 = -", minus_cols)

# 计算 总收入_calc
df_num = df.copy()
for c in set(plus_cols + minus_cols + ["本金"]):
    if c in df_num.columns:
        df_num[c] = df_num[c].map(to_num)

df_num["总收入_calc"] = 0.0
if plus_cols:
    df_num["总收入_calc"] += df_num[plus_cols].sum(axis=1, skipna=True)
if minus_cols:
    df_num["总收入_calc"] -= df_num[minus_cols].sum(axis=1, skipna=True)

# --- 3.2 复现“日策略内部计算年化收益率” V列：=POWER(总收入/本金+1,365)-1 ---
df_num["日策略内部计算年化收益率_calc"] = np.power(df_num["总收入_calc"] / df_num["本金"] + 1.0, 365) - 1.0

# --- 3.3 复现“日收益” W列：=本金 * 日策略内部计算年化收益率 / 365 ---
df_num["日收益_calc"] = df_num["本金"] * df_num["日策略内部计算年化收益率_calc"] / 365.0

# 展示对照：你会看到原“日收益”是NaN，但 calc 有值（这就是根因）
display(df_num[["日期","本金","总收入","总收入_calc","日策略内部计算年化收益率","日策略内部计算年化收益率_calc","日收益","日收益_calc"]].head(30))


shape: (221, 38)
columns contains 日收益? True


,日期,日收益,总收入,本金
0,2025-06-23 00:00:00,NaN,NaN,1000000.0
1,2025-06-24 00:00:00,NaN,NaN,1000000.0
2,2025-06-25 00:00:00,NaN,NaN,1000000.0
3,2025-06-26 00:00:00,NaN,NaN,1000000.0
4,2025-06-27 00:00:00,NaN,NaN,1000000.0
5,2025-06-28 00:00:00,NaN,NaN,1000000.0
6,2025-06-29 00:00:00,NaN,NaN,1000000.0
7,2025-06-30 00:00:00,NaN,NaN,1000000.0
8,2025-07-01 00:00:00,NaN,NaN,1000000.0
9,2025-07-02 00:00:00,NaN,NaN,1000000.0


col(日收益/总收入/本金/日策略内部计算年化收益率): 23 20 15 22
row 4 日收益 cell raw = =O4*V4/365
总收入公式示例: SUM(B4:G4)-I4+SUM(J4:K4)
总收入 = + ['单所bn资费', '单所bnTt价差', '单所bnTm价差', '单所bn借贷', '单所okx资费', '单所okx价差', '跨所双合约资费', '跨所双合约价差']
总收入 = - ['单所okx借贷']


,日期,本金,总收入,总收入_calc,日策略内部计算年化收益率,日策略内部计算年化收益率_calc,日收益,日收益_calc
0,2025-06-23 00:00:00,1000000.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
1,2025-06-24 00:00:00,1000000.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
2,2025-06-25 00:00:00,1000000.0,NaN,79.929099,NaN,0.029603,NaN,81.103162
3,2025-06-26 00:00:00,1000000.0,NaN,258.041594,NaN,0.098750,NaN,270.547530
4,2025-06-27 00:00:00,1000000.0,NaN,290.317412,NaN,0.111767,NaN,306.210456
5,2025-06-28 00:00:00,1000000.0,NaN,404.848567,NaN,0.159211,NaN,436.195224
6,2025-06-29 00:00:00,1000000.0,NaN,350.624496,NaN,0.136502,NaN,373.979284
7,2025-06-30 00:00:00,1000000.0,NaN,449.352014,NaN,0.178187,NaN,488.183013
8,2025-07-01 00:00:00,1000000.0,NaN,347.274182,NaN,0.135114,NaN,370.175298
9,2025-07-02 00:00:00,1000000.0,NaN,162.999771,NaN,0.061295,NaN,167.932110


In [ ]:
df_num['总收入']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
216   NaN
217   NaN
218   NaN
219   NaN
220   NaN
Name: 总收入, Length: 221, dtype: float64